In [26]:
# %matplotlib inline
%matplotlib widget

In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# IMPORT

In [28]:
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import xarray as xr


from pycns import CnsStream, CnsReader, get_viewer
import physio

from physio_rea_analyse import *
from custom_view import *

from configuration import *
from tools import *
from params import *

from icp_jobs import *

# READER

In [29]:
subs

['Patient_2024_May_16__9_33_08_427295',
 'Patient_2024_May_25__16_48_44_716049',
 'Patient_2024_May_25__16_09_03_881877',
 'Patient_2024_Jun_24__12_33_38_160174',
 'PatientData_1723890119868125']

In [30]:
patient_id = 'Patient_2024_May_16__9_33_08_427295'
# patient_id = 'Patient_2024_May_25__16_48_44_716049'
# patient_id = 'Patient_2024_May_25__16_09_03_881877'
# patient_id = 'Patient_2024_Jun_24__12_33_38_160174'
# patient_id = 'PatientData_1723890119868125'

raw_folder = data_path / patient_id
cns_reader = CnsReader(raw_folder, with_events=True, event_time_zone='Europe/Paris')

In [31]:
meta = get_metadata(patient_id)
meta

Numero                              2
Sexe                                F
Age                                59
Date_admission    2024-05-02 00:00:00
Date_sortie       2024-07-18 00:00:00
Duree_sejour                       78
Indication                        2.5
Date_pose         2024-05-02 00:00:00
Date_ablation     2024-05-17 00:00:00
Duree_DVE                          15
Kakarla                           1.0
EIDVE                               0
DVI                                 0
Date_DVI                          NaT
Delais_DVI                        NaN
VVTDM0                            NaN
VVTDM1                            NaN
VVTDM2                            NaN
VVTDM3                            NaN
VSTDM1                            NaN
VSTDM2                            NaN
VVECHO1                           NaN
VVECHO2                           NaN
VSECHO1                           NaN
VSECHO2                           NaN
PICPO                             NaN
PICPA       

In [32]:
pd.DataFrame(cns_reader.events)

,name,start_time,duration,description


# PRECOMPUTE

In [33]:
# force_compute = False

# if force_compute:
#     icp_stream = cns_reader.streams['ICP']
#     srate_icp = icp_stream.sample_rate
#     raw_icp, dates = icp_stream.get_data(with_times=True, apply_gain=True)
#     icp_features = compute_icp(raw_icp, srate_icp, date_vector = dates)
# else:
#     icp_features = detect_icp_job.get(patient_id).to_dataframe()
# print(icp_features.shape)

# VIEWER RAW

In [34]:
cns_reader.streams.keys()

dict_keys(['SpO2', 'NBP_Dias', 'RR', 'P2_Mean', 'HR', 'P2_Syst', 'ECG_II', 'P1_Dias', 'ABP', 'NBP_Mean', 'P1_Mean', 'CO2', 'P1', 'P1_Syst', 'EtCO2_Ext', 'NBP_Syst', 'P2_Dias', 'PLETH', 'P2', 'RESP'])

In [35]:
stream_names = list(cns_reader.streams.keys())
stream_with_P = [name for name in stream_names if 'P' in name and len(name) < 4]
stream_with_P

['ABP', 'P1', 'P2']

In [36]:
w = get_viewer(cns_reader, stream_names=stream_with_P, with_events = False)
w

Viewer(children=(VBox(children=(HBox(children=(Button(description='autoscale', icon='refresh', style=ButtonSty…

# VIEWER EXTERNALS

In [37]:
icp_chan_name[patient_id]

'P1'

In [38]:
icp_features = detect_icp_job.get(patient_id).to_dataframe()
stream_name = icp_chan_name[patient_id]
psi_da = psi_job.get(patient_id)['psi']

ext_plots = {
            'detections':ICP_Detections(cns_reader.streams[stream_name], icp_features),
            'psi':PSI(psi_da)
}

w = get_viewer(cns_reader, stream_names=[stream_name], ext_plots=ext_plots)
w

Viewer(children=(VBox(children=(HBox(children=(Button(description='autoscale', icon='refresh', style=ButtonSty…